In [1]:
import os

In [2]:
from datetime import datetime
from collections import deque
import time as t

In [3]:
from cassiopeia import riotapi
from cassiopeia.type.api.exception import APIError
from cassiopeia.type.core.common import LoadPolicy
from set_up import set_up

In [4]:
set_up('EUNE')

In [5]:
def auto_retry(api_call_method):
    """ A decorator to automatically retry 500s (Service Unavailable)
    and skip 400s (Bad Request) or 404s (Not Found). """
    def call_wrapper(*args, **kwargs):
        try:
            return api_call_method(*args, **kwargs)
        except APIError as error:
            # Try Again Once
            if error.error_code in [500]:
                try:
                    print("Got a 500, trying again...")
                    return api_call_method(*args, **kwargs)
                except APIError as another_error:
                    if another_error.error_code in [500, 400, 404]:
                        pass
                    else:
                        raise another_error

            # Skip
            elif error.error_code in [400, 404]:
                print("Got a 400 or 404")
                pass

            # Fatal
            else:
                raise error
    return call_wrapper

In [6]:
riotapi.get_match = auto_retry(riotapi.get_match)
riotapi.get_summoner_by_id = auto_retry(riotapi.get_summoner_by_id)
riotapi.get_summoner_by_name = auto_retry(riotapi.get_summoner_by_name)

In [ ]:
%%time
unpulled_summoners = deque(entry.summoner for entry in riotapi.get_challenger())

In [9]:
pulled_summoners = deque()

In [10]:
gather_start = datetime(2017, 7, 9)  # 1 day after patch 5.14
matchList = open('MatchList.txt', 'a+')

In [12]:
%%time
while len(unpulled_summoners) > 101:
    summoner = unpulled_summoners.popleft()
    for match_reference in summoner.match_list(begin_time=gather_start):
        continue
        # If you are connected to a database, the match will automatically be stored in it.
        # Simply pull the match, and it's in your database for whenever you need it again!
        # If you pull a match twice, the second time it will be loaded from the database rather than pulled from Riot
        # and therefore will not count against your rate limit. This is true of all datatypes, include Summoner.
        match = riotapi.get_match(match_reference)
        if match is None:  # If the match still fails to load, continue on to the next one
                continue
        elif str(match) in matchList.read():
            continue
        else:
            matchList.write(str(match))
        print("Stored {0} in my database".format(match))
        for participant in match.participants:
            if participant.summoner not in unpulled_summoners and participant.summoner not in pulled_summoners:
                    unpulled_summoners.append(participant.summoner)
    pulled_summoners.append(summoner)
print(pulled_summoners)

APIError: Server returned error 503 on call: https://eune.api.pvp.net/api/lol/eune/v2.2/matchlist/by-summoner/41649116?beginTime=1499558400000&api_key=RGAPI-4915d3b4-48d6-41a7-b561-82abf1a28909